In [ ]:
import numpy as np 
import pandas as pd

In [2]:
!pip install -q transformers accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.1/566.1 kB 11.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 98.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 69.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 33.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 30.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 1.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 83.0 MB/s eta 0:00:00:00:0100:01
ERROR: pip's d

In [3]:
!pip install -U bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 30.9 MB/s eta 0:00:00:00:0100:01


In [4]:
import torch
import requests
from PIL import Image
from io import BytesIO
from transformers import AutoProcessor, Qwen2VLForConditionalGeneration, BitsAndBytesConfig

model_path = "Qwen/Qwen2-VL-2B-Instruct"

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type='nf4'
)

model =Qwen2VLForConditionalGeneration.from_pretrained(
    model_path,
    torch_dtype=torch.float16,
    device_map="auto",
    quantization_config=quantization_config
)

processor = AutoProcessor.from_pretrained(model_path)

2025-11-01 04:51:19.041595: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1761972679.272039      37 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1761972679.333126      37 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


config.json: 0.00B [00:00, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/429M [00:00<?, ?B/s]

...

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

You have video processor config saved in `preprocessor.json` file which is deprecated. Video processor configs should be saved in their own `video_preprocessor.json` file. You can rename the file or load and save the processor back which renames it automatically. Loading from `preprocessor.json` will be removed in v5.0.


chat_template.json: 0.00B [00:00, ?B/s]

In [ ]:
def caption_images_batch(image_files_list, prompt):
    """
    Generates captions for a BATCH of image files (URLs or local paths)
    using the loaded Qwen-VL model and a single shared prompt.
    """
    images = []
    max_size = (512, 512) 
    
    for image_file in image_files_list:
        try:
            if image_file.startswith('http') or image_file.startswith('https'):
                response = requests.get(image_file, timeout=10) 
                current_image = Image.open(BytesIO(response.content)).convert('RGB')
            else:
                current_image = Image.open(image_file).convert('RGB')
            
            current_image.thumbnail(max_size, Image.Resampling.LANCZOS)
            images.append(current_image)
            
        except Exception as e:
            print(f"Error loading image {image_file}: {e}")
            images.append(None) 

    
    valid_images = [img for img in images if img is not None]
    
    if not valid_images:
        return ["Error: No valid images in batch"] * len(image_files_list)

    conversations = []
    for _ in valid_images:
        conversations.append([
            {
                "role": "user",
                "content": [
                    {"type": "image"},
                    {"type": "text", "text": prompt}
                ]
            }
        ])

    text_prompts = processor.apply_chat_template(conversations, add_generation_prompt=True, tokenize=False)
    
    inputs = processor(
        text=text_prompts,
        images=valid_images,
        return_tensors="pt",
        padding=True 
    ).to(model.device)

    with torch.inference_mode():
        output_ids = model.generate(
            **inputs, 
            max_new_tokens=512,
            do_sample=False
        )

    input_token_len = inputs['input_ids'].shape[1]
    generated_ids = output_ids[:, input_token_len:]
    responses = processor.batch_decode(generated_ids, skip_special_tokens=True)
    
    cleaned_responses = [resp.strip() for resp in responses]
    
    final_output = []
    resp_index = 0
    for img in images:
        if img is None:
            final_output.append("Error: Image load failed")
        else:
            final_output.append(cleaned_responses[resp_index])
            resp_index += 1
            
    return final_output

In [ ]:
# import pandas as pd
import time
import math

BATCH_SIZE = 36
YOUR_CSV_FILE = '/kaggle/input/kbfkwebfiwe/test_dataset.csv'
OUTPUT_CSV_FILE = '/kaggle/working/generated_captions_test.csv'
MEDIA_COLUMN_NAME = 'media_url'
NO_MEDIA_STRING = "no media found"

text_prompt="""
Describe everything visible in the image with maximum objective detail, including all objects, environment, people, text, vehicle features, materials, lighting, spatial layout, textures, and reflections without guessing unseen context or intent. give """

print("Loading dataset from {YOUR_CSV_FILE}...")
try:
    df = pd.read_csv(YOUR_CSV_FILE,nrows=1000)
except FileNotFoundError:
    print(f"Error: File not found at {YOUR_CSV_FILE}")

df['generated_caption'] = None
print(f"Dataset loaded. Total rows: {len(df)}")

total_batches = math.ceil(len(df) / BATCH_SIZE)
total_start_time = time.time()

print(f"Starting processing in batches of {BATCH_SIZE}...")

for i in range(0, len(df), BATCH_SIZE):
    batch_start_time = time.time()
    
    batch_df = df.iloc[i : i + BATCH_SIZE]
    
    urls_to_process = []
    valid_indices = []

    for index, row in batch_df.iterrows():
        media_data = str(row["media_url"])
        
        if NO_MEDIA_STRING not in media_data.lower() and pd.notna(media_data):
            extracted_url = media_data
            if extracted_url:
                urls_to_process.append(extracted_url)
                valid_indices.append(index) # यह इंडेक्स (e.g., 20, 21, 23, 27...)
            else:
                df.at[index, 'generated_caption'] = "Error: URL extraction failed"
        else:
            df.at[index, 'generated_caption'] = ""

    if urls_to_process:
        print(f"Processing Batch {i//BATCH_SIZE + 1}/{total_batches} (Found {len(urls_to_process)} valid images)...")
        
        batch_captions = caption_images_batch(urls_to_process, text_prompt)

        for j, caption in enumerate(batch_captions):
            original_index = valid_indices[j]
            df.at[original_index, 'generated_caption'] = caption
    else:
        print(f"Skipping Batch {i//BATCH_SIZE + 1}/{total_batches} (No valid images found)")

    batch_end_time = time.time()
    print(f"Batch {i//BATCH_SIZE + 1} finished in {batch_end_time - batch_start_time:.2f} seconds.")

    if (i // BATCH_SIZE) % 5 == 0:
        print(f"Saving intermediate results to {OUTPUT_CSV_FILE}...")
        df.to_csv(OUTPUT_CSV_FILE, index=False)


total_end_time = time.time()
print("\n--- Processing Complete ---")
print(f"Total time taken: {(total_end_time - total_start_time) / 60:.2f} minutes.")
print(f"Saving final results to {OUTPUT_CSV_FILE}...")
df.to_csv(OUTPUT_CSV_FILE, index=False)

print("Done.")

Loading dataset from {YOUR_CSV_FILE}...
Dataset loaded. Total rows: 1000
Starting processing in batches of 36...
Processing Batch 1/28 (Found 36 valid images)...
Error loading image https://pbs.twimg.com/media/EXaIreiXsAAKRuu?format=jpg&name=small: cannot identify image file <_io.BytesIO object at 0x7f4a2e20e1b0>
Error loading image https://pbs.twimg.com/media/ElB41cOW0AETKcX?format=jpg&name=small: cannot identify image file <_io.BytesIO object at 0x7f49fabc7dd0>


The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Batch 1 finished in 81.31 seconds.
Saving intermediate results to /kaggle/working/generated_captions_test.csv...
Processing Batch 2/28 (Found 36 valid images)...
Error loading image https://pbs.twimg.com/media/EA47wp5XsAALqO8?format=jpg&name=small: cannot identify image file <_io.BytesIO object at 0x7f4a2e20e1b0>
Error loading image https://pbs.twimg.com/media/DmkhTgJWwAAEts7?format=jpg&name=small: cannot identify image file <_io.BytesIO object at 0x7f4a2e1c2f70>
Error loading image https://pbs.twimg.com/media/EWLW38GU0AAOLyA?format=jpg&name=small: cannot identify image file <_io.BytesIO object at 0x7f4a2e20d940>


The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Batch 2 finished in 96.71 seconds.
Processing Batch 3/28 (Found 36 valid images)...
Error loading image https://pbs.twimg.com/ext_tw_video_thumb/1152653784419622912/pu/img/bn1si8rNAaZAAFJo.jpg: cannot identify image file <_io.BytesIO object at 0x7f4a3ce932e0>


...

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Batch 11 finished in 97.30 seconds.
Saving intermediate results to /kaggle/working/generated_captions_test.csv...
Processing Batch 12/28 (Found 36 valid images)...
Error loading image https://pbs.twimg.com/media/EETTKm0UUAEQN1U?format=jpg&name=small: cannot identify image file <_io.BytesIO object at 0x7f4a2e32afc0>
Error loading image https://pbs.twimg.com/media/EUfzh-zU0AAtCIo?format=jpg&name=small: cannot identify image file <_io.BytesIO object at 0x7f4a3c38c900>


The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Batch 12 finished in 76.75 seconds.
Processing Batch 13/28 (Found 36 valid images)...
Error loading image https://pbs.twimg.com/media/D8YW6XOWwAAWoYe?format=png&name=small: cannot identify image file <_io.BytesIO object at 0x7f4a3e7c89f0>
Error loading image https://pbs.twimg.com/media/EBjiH6AWkAUkjkn?format=jpg&name=small: cannot identify image file <_io.BytesIO object at 0x7f4a3ce93ba0>
Error loading image https://pbs.twimg.com/media/EpNG-yOXEAMl8_5?format=jpg&name=small: cannot identify image file <_io.BytesIO object at 0x7f4a3ce3f4c0>


The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
